### imports

In [1]:
from modules.models import define_VAE, define_MM
from modules.variables import HDVariables
from keras.utils import plot_model
import h5py
import numpy as np
import keras

Using TensorFlow backend.


### get variables from hard drive

Variables are saved when training the MCVAE, and all are accessed via the 
HDVariables class.

In [2]:
hd_var = HDVariables()

### define best VAE weights path

In [3]:
vae_weights_path = "weights/" + "test_run-99-0.80.hdf5"

### define VAE and set weights

In [6]:
# define model
vae_full, vae_enc, vae_dec = define_VAE(enc_units = int(hd_var.enc_units),
                                        dec_units = int(hd_var.dec_units),
                                        latent_units = int(hd_var.latent_units),
                                        timesteps = int(hd_var.timesteps),
                                        features = int(hd_var.features),
                                        dropout = float(hd_var.dropout),
                                        beta = float(hd_var.beta),
                                        learning_rate = float(hd_var.learning_rate),
                                        epsilon_std = float(hd_var.epsilon_std))

vae_full.load_weights(vae_weights_path)
print("Loaded VAE weights from disk")

# get all weights
all_weights = vae_full.get_weights()

# find encoder weights position
decoder_position = len(vae_enc.get_weights())
decoder_weights = all_weights[decoder_position::]
encoder_weights = all_weights[:decoder_position]

# set encoder weights
vae_enc.set_weights(encoder_weights)
vae_dec.set_weights(decoder_weights)
print("Set VAE weights")

Loaded VAE weights from disk
Set VAE weights


### setup GPU 

In [4]:
import os
import tensorflow as tf
import keras.backend as K

# Only for GPU use:
os.environ["CUDA_VISIBLE_DEVICES"]="1"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)

# Predict VAE sequence

In [47]:
from modules.predictions import predict_VAE_sequence
from modules.data_prep import VAEdataPrep

dataPrep = VAEdataPrep(int(hd_var.timesteps), int(hd_var.features))

# for reproducability
np.random.seed(10)


# test set 
test_IDs = []
ID_list = np.load(str(hd_var.generator_IDs_file))
test_list = ID_list.item().get("test")


input_songs = []
output_songs = []

ID = test_list[20]
encoder_input_tmp = np.load(str(hd_var.encoder_dir) + "/" + ID + '.npy')
encoder_input = encoder_input_tmp.reshape(1, int(hd_var.timesteps), int(hd_var.features))

encoder_output = predict_VAE_sequence(infenc = vae_enc,
                                  infdec = vae_dec,
                                  source = encoder_input,
                                  seq_len = 400,
                                  timesteps = int(hd_var.timesteps),
                                  n_decoder_units = int(hd_var.dec_units),
                                  latent_dim = int(hd_var.latent_units),
                                  temperature = 1)

input_songs.append(dataPrep.one_hot_decode(encoder_input_tmp))
output_songs.append(dataPrep.one_hot_decode(encoder_output))

In [26]:
print("Input sequence")
print(np.ravel(input_songs))

print("\n\n","Predicted sequence")
print(np.ravel(output_songs))

Input sequence
[ 36 129 129 129 129 128  36 128  31 129 129 128  36 129 129 128]


 Predicted sequence
[129 129 129 129  31 129 129 128 129 129 129 129 128 129 128 129  33 129
 128 129 128 129  38 129  31 129 129 128 129 128 129  26 129 128 128  26
 128 129 128 129  42 129 128 129 128 129  30 128 128 129  42 128  16 128
 116 128 128  12 128  32 128 122 128 123 128  64 128  35 128  14 128  99
  33  31 128 128 110  86 128  11 128 123 128  35  78 128  95 128  54 128
 115  78  47 128  95 128 128  97 128  41 128  30 128  42 128  86 128 114
 128  24 128 116 128 110 128  39 128  24 128  12 128  42 128  22 128 122
 128  13 128  14 128  83 128  82 128  78 128 109 128  63 128  30 128  61
 128   3 128  15 128  43 128  49 128  44 128  72 128  84 128  51 128 118
 128  69 128  68 128  57 128  62 128  37 128  87 128  84 128  33 128  79
 128  96 128  60 128 126  70 128  57 128  70 128 128  69 128  61 128  39
 128  54 128  44 128  66 128  60 128  73 128  70 128  42 128  67 128  74
 128  93 128  78 128 

# Predict song with VAE

In [37]:
np.random.seed(10)

# song set 
song_IDs = []
ID_list = np.load(str(hd_var.songs_ID_file))
test_list = ID_list.item().get("test")

ID = test_list[20]
song = np.load(str(hd_var.encoder_dir_songs) + "/" + ID + '.npy')

orig_song = []
pred_song = []
    
for bar in song:
    encoder_input = bar.reshape(1, int(hd_var.timesteps), int(hd_var.features))
    encoder_output = predict_VAE_sequence(infenc = vae_enc,
                                  infdec = vae_dec,
                                  source = encoder_input,
                                  seq_len = int(hd_var.timesteps),
                                  timesteps = int(hd_var.timesteps),
                                  n_decoder_units = int(hd_var.dec_units),
                                  latent_dim = int(hd_var.latent_units),
                                  temperature = 1)


    orig_song.append(dataPrep.one_hot_decode(encoder_input.reshape(int(hd_var.timesteps), int(hd_var.features))))
    pred_song.append(dataPrep.one_hot_decode(encoder_output.reshape(int(hd_var.timesteps), int(hd_var.features))))

In [40]:
print("Original song")
#print(np.ravel(orig_song))

print("\n\n","Predicted song")
#print(np.ravel(pred_song))

Original song


 Predicted song


### define MDN decoder

In [43]:
from modules.models import define_MM
_, mdn_dec = define_MM(seq_len = int(hd_var.mdn_seq_len),
                    vae_latent_units = int(hd_var.latent_units),
                    hidden_units = int(hd_var.mdn_hidden_units),
                    number_mixtures = int(hd_var.number_mixtures))

# Predict song with MCVAE

In [60]:
from modules.mdn import sample_from_output
from modules.predictions import predict_z, mdn_predict_sequence

# test set 
test_IDs = []
ID_list = np.load(str(hd_var.generator_IDs_file))
test_list = ID_list.item().get("test")

ID = test_list[20]
encoder_input_tmp = np.load(str(hd_var.encoder_dir) + "/" + ID + '.npy')
encoder_input = encoder_input_tmp.reshape(1, int(hd_var.timesteps), int(hd_var.features))


z = predict_z(infenc = vae_enc, source = encoder_input, latent_dim=int(hd_var.latent_units))

mdn_song = []

for i in range(25):

    params = mdn_dec.predict(z.reshape(1,1,int(hd_var.latent_units)))
    z = sample_from_output(params[0], int(hd_var.latent_units), int(hd_var.number_mixtures), temp=1)
    song = mdn_predict_sequence(infdec = vae_dec,
                                 z = z,
                                 latent_dim = int(hd_var.latent_units),
                                 seq_len = int(hd_var.mdn_seq_len),
                                 temperature = 0.2,
                                 n_decoder_units = int(hd_var.dec_units))

    mdn_song.append(dataPrep.one_hot_decode(song.reshape(int(hd_var.mdn_seq_len), int(hd_var.features))))

In [61]:
print("Input sequence")
print(np.ravel(dataPrep.one_hot_decode_song(encoder_input)))

print("\n\n","Predicted sequence")
print(np.ravel(mdn_song))

Input sequence
[ 36 129 129 129 129 128  36 128  31 129 129 128  36 129 129 128]


 Predicted sequence
[124 119  76  67 111  96   6  17   8   2  83 113 120 110  17  35 122  39
 109 108 112 107 108 100  28  99 108  44  20 120  48  60 113  56  27 125
  47  76  38 117  34  28   1  32 100  41  60  21  49  86  84 122  16  13
 120  17  43  74 130 111  67  33  44  23 116  86   0  70 120  42  68  70
  44  32   0  42  38  38  70  51   3  29  57 106  13  13  95  82  71 122
 104  63  44 118  43  66  76  72  20 102  72  30  46   6 118 118  82 123
  33  32  57 114 125  44 101  50  20  58  89  79  72   5  62  42   8  70
   2   4  42 112  61  38   1  25  63  97  25  82 108  87  38  60  40 101
  11  33 129 124  31 110  84 109  54 117  22   8 101  95  78  84  32  59
 130 112  89  95  68  37  98  59 122 100 122  89 129  43 124  13  63   7
 119  40 116  15  24  73  40 113 116  76  17 103  13 123  29 109  35  55
  58  39  58  92  34  38  18 100  78  28  82  88   0  76 115  43 119  57
  89  75  90 103  52 

# Last step

This is where the MCVAE gets z-1 and predicts z.
Will be finished in the end of January.